In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='-1' #use cpu

# imports

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


print(f"{tf.__version__ = }")

tf.__version__ = '2.6.1'


In [2]:
n_gpus = len(tf.config.list_physical_devices("GPU"))
print(f"{n_gpus = }")


n_gpus = 0


2022-03-02 01:35:21.373507: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-03-02 01:35:21.373575: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (583c5dd25dec): /proc/driver/nvidia/version does not exist


## 1. Classifying movie reviews

In [6]:
DIM=5_000 #10_000 bricks my laptop :(

In [7]:
# load pre-processed imdb

from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=DIM)

In [8]:
print(f"{len(train_data[0]) = }")
print(f"{train_data[0][:10] = }")
print(f"{train_labels[0] = }")

len(train_data[0]) = 218
train_data[0][:10] = [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]
train_labels[0] = 1


In [9]:
for i in range(5):
    sample_len = len(train_data[i])
    print(f"{i = }: {sample_len = }")

i = 0: sample_len = 218
i = 1: sample_len = 189
i = 2: sample_len = 141
i = 3: sample_len = 550
i = 4: sample_len = 147


In [10]:
import numpy as np
from typing import List

def multi_hot_encode(sequences: List[List], dimension: int = 10_000):
    results = np.zeros(shape=(len(sequences), dimension))

    for i, seq in enumerate(sequences):
        results[i, seq] = 1.
    
    return results


In [11]:
x_train = multi_hot_encode(train_data, dimension=DIM)
x_test = multi_hot_encode(test_data, dimension=DIM)

In [12]:
# also vectorize labels
y_train = np.asarray(train_labels).astype("float32") 
y_test = np.asarray(test_labels).astype("float32") 

In [13]:
# build 3 layer model
model = keras.Sequential(
    [
        layers.Dense(16, activation="relu"),
        layers.Dense(16, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

2022-03-02 01:35:59.071269: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
model.compile(
    optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [15]:
# create validation set

x_val = x_train[:10_000]
partial_x_train = x_train[10_000:]
y_val = y_train[:10_000]
partial_y_train = y_train[10_000:]

In [16]:
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=20,
    batch_size=64,
    validation_data=(x_val, y_val),
)